In [89]:
from dataclasses import dataclass
import math

@dataclass(frozen=True)
class Vector3:
    x: float
    y: float
    z: float

    # Vector addition
    def __add__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    # Vector subtraction
    def __sub__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )

    # Scalar multiplication
    def __mul__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x * scalar,
            self.y * scalar,
            self.z * scalar
        )
    
    # Scalar division
    def __truediv__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x / scalar,
            self.y / scalar,
            self.z / scalar
        )

    __rmul__ = __mul__
    __rtruediv__ = __truediv__

    # Dot product
    def dot(self, other: "Vector3") -> float:
        return (
            self.x * other.x +
            self.y * other.y +
            self.z * other.z
        )

    # Cross product
    def cross(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )

    # Magnitude (length)
    def magnitude(self) -> float:
        return math.sqrt(self.dot(self))

    # Unit vector
    def unit_vector(self) -> "Vector3":
        mag = self.magnitude()
        if mag == 0:
            raise ValueError("Cannot normalize a zero vector")
        return self * (1 / mag)

In [90]:
# Semi Major Axis (a)
def compute_sma(mu:float, energy:float):
    return -mu/(2*energy)


In [91]:
# Eccentricity (e)
import math

def compute_ecc(energy: float, mu: float, h:float):
    return math.sqrt(1+(2*math.pow(h, 2)*energy)/(math.pow(mu, 2)))

In [92]:
# Inclination (i)
import math

def compute_inc(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return math.acos(Z_.dot(h_))


In [93]:
# RAAN (capital omega) Ω
import math

def compute_raan(Nx: float, Ny: float):
    return math.atan2(Ny, Nx)

In [94]:
# Argument of Perigeee (omega sub p) ωp
import math

def compute_argp(h_: Vector3, N_: Vector3, B_: Vector3):
    h_
    N_
    B_
    return math.atan2(h_.dot(N_.cross(B_)), N_.dot(B_))



In [95]:
# True Anomaly (nu) ν
import math

def compute_ta(v_: Vector3, r_: Vector3, B_: Vector3):
    cosTa = r_.dot(B_)/(r_.magnitude()*B_.magnitude())
    ta = math.acos(cosTa)
    if r_.dot(v_) < 0:
        ta = 2*math.pi - ta
    return ta

In [96]:
# Period (TP)
import math

def compute_period(a: float, mu: float):
    return 2*math.pi*math.sqrt(math.pow(a, 3)/mu)

In [97]:
# Apogee (r sub a) ra
def compute_apogee(a: float, e: float):
    return a*(1+e)

In [98]:
# Perigee (r sub p) rp
def compute_perigee(a: float, e: float):
    return a*(1-e)

In [99]:
# Energy (xi) ξ
import math

def compute_energy(v: float, r: float, mu: float):
    return math.pow(v, 2)/2 - mu/r

In [100]:
# Angular Momentum Vector (h_)
def compute_angular_momentum_vector(r: Vector3, v: Vector3):
    return r.cross(v)

In [101]:
# RAAN Vector (N_) Unitized
def compute_raan_vector(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return (Z_.cross(h_)/(Z_.cross(h_).magnitude())).unit_vector()

In [102]:
# Perigee Vector (B_) Unitized
def compute_perigee_vector(mu: float, r_: Vector3, v_: Vector3, h_: Vector3):
    return v_.cross(h_)-mu*(r_/r_.magnitude()).unit_vector()

In [103]:
import math

class KeplerianElements:
    """
    Computes classical Keplerian orbital elements from
    position and velocity state vectors.
    """

    mu_earth = 3.986004418e14  # m^3 / s^2

    def __init__(self, position: Vector3, velocity: Vector3):
        """
        Parameters
        ----------
        position : Vector3
            Position vector (meters)
        velocity : Vector3
            Velocity vector (m/s)
        """

        mu = self.mu_earth
        k_hat = Vector3(0, 0, 1)

        # Specific orbital energy
        self.xi = compute_energy(velocity.magnitude(), position.magnitude(), mu)

        # Semi-major axis
        self.a = compute_sma(mu, self.xi)

        # Angular momentum vector
        self.h_ = compute_angular_momentum_vector(position, velocity)

        # Eccentricity
        self.ecc = compute_ecc(self.xi, mu, self.h_.magnitude())

        # Inclination
        self.inc = compute_inc(k_hat, self.h_)
        self.inc_deg = math.degrees(self.inc)

        # RAAN
        self.N_Unit = compute_raan_vector(k_hat, self.h_)
        self.raan = compute_raan(self.N_Unit.x, self.N_Unit.y)
        self.raan_deg = math.degrees(self.raan)

        # Argument of perigee
        self.B_Unit = compute_perigee_vector(
            mu,
            position,
            velocity,
            self.h_
        )
        self.argp = compute_argp(self.h_, self.N_Unit, self.B_Unit)
        self.argp_deg = math.degrees(self.argp)

        # True anomaly
        self.ta = compute_ta(velocity, position, self.B_Unit)
        self.ta_deg = math.degrees(self.ta)

        # Orbital period (elliptical only)
        self.period = compute_period(self.a, mu)

        # Apoapsis / periapsis
        self.apogee = compute_apogee(self.a, self.ecc)
        self.perigee = compute_perigee(self.a, self.ecc)


```^ Previous Lessons ^```

In [104]:
# Problem 1

pos = Vector3(326151.080726, 6077471.251787, 2944583.918767)  # meters
vel = Vector3(-7455.178720, -482.482572, 1910.883434) # m/s

kepler_elements = KeplerianElements(pos, vel)
print("nu (true anomaly) is: " + str(kepler_elements.ta) + " rad")

nu (true anomaly) is: 0.533708002792791 rad


In [105]:
import math

# compute eccentric anomaly (E)
def compute_eccentric_anomaly(nu: float, e: float) -> float:
    return math.asin((math.sin(nu)*math.sqrt(1 - math.pow(e, 2))) / (1 + e*math.cos(nu)))


In [106]:
E = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
print("E0 (eccentric anomaly) is: " + str(E) + " rad")

E0 (eccentric anomaly) is: 0.5286424011417852 rad


In [107]:
import math

# compute mean anomaly (M)
def compute_mean_anomaly(E: float, e: float) -> float:
    return E - e*math.sin(E)

In [108]:
M = compute_mean_anomaly(E, kepler_elements.ecc)
print("M0 (mean anomaly) is: " + str(M) + " rad")

M0 (mean anomaly) is: 0.5235987858960514 rad
